# Mid Term 2
*Authors*: Aliotta Lorenzo & Loris Prataiolo

# *Abstract*
This project aims to enstablish the optimal hyperparameters for the Kernel Regularized Least Squares (KRLS) Algorithm: Regularizator parameter & Kernel parameter. The process to obtain those parameters is driven by the Q-Cross Validation Algorithm (`krls_kfold_valerr`), over a real binary dataset  of which was not given a clue about the contex of data. After enstablished the regularizator and the Kernel parameter, is defined the test section, which make it is possible to perform the model over a test set with the pairs of vaules identified.

# KRLS Algorithm
Given kernel $K = k(x_i, x_j)$ for $i=(1, \dots, n)$ and $j=(1, \dots, n)$, KRLS learns a weight-vector with the following formula

$$(K + n \lambda I)w = Y$$

and then predictions on some new point $\tilde{x}$ are given by

$$f^{\mathrm{KRLS}}(\tilde{x}) = k(\tilde{x}, X^{\mathrm{train}}) w$$

In [12]:
%matplotlib inline
import numpy as np
from numpy.random import Generator
import matplotlib.pyplot as plt
import scipy.spatial


def _check_random_generator(rng: int | Generator | None) -> Generator:
    """Convert rng into a np.random.Generator instance."""
    if rng is None:
        print("⚠️  Warning: Set the `rng` parameter to a NumPy random generator\n"
              "   (e.g. `np.random.default_rng(42)`) or specify a fixed seed to ensure\n"
              "   your results are reproducible. Proceeding with a random seed for now.")
        return np.random.default_rng()
    if isinstance(rng, np.random.Generator):
        return rng
    if isinstance(rng, (int, np.integer)):
        return np.random.default_rng(rng)

# Optimal Hyperparameters Identification Process

In [3]:
X_train = np.load('X_train.npy')
Y_train = np.load('y_train.npy')

print(f'Shapes: \n\t - Feature shape: {X_train.shape} \n \t - Binary Labels shape: {Y_train.shape}')

Shapes: 
	 - Feature shape: (720, 7) 
 	 - Binary Labels shape: (720,)


# Kernels

In [5]:
def squared_distances(X1, X2):
    """Compute the matrix of pairwise squared-distances between all points in X1 and in X2.
    """
    return scipy.spatial.distance.cdist(X1, X2, metric='seuclidean')

In [6]:
def linear_kernel (X, X_prime):
    return X @ X_prime.T

def poly_kernel(X, X_prime, exponent):
    return np.pow((X @ X_prime.T + 1), exponent)

def gaussian_kernel(X, X_prime, lengthscale):
    exp_num   = squared_distances(X, X_prime)
    exp_denom = 2* np.pow(lengthscale, 2)
    exponent  = exp_num / exp_denom
    return np.exp(-exponent)

In [7]:
def kernel_matrix(X1, X2, kernel_type, param):
    # X1 : array of shape n x d
    # X2 : array of shape m x d
    if kernel_type == 'linear':
        return linear_kernel(X1,X2)
        
    elif kernel_type == 'polynomial':
        exponent = param
        return poly_kernel(X1, X2, exponent)
        
    elif kernel_type == 'gaussian':
        lengthscale = param
        return gaussian_kernel(X1, X2, lengthscale)
    else:
        raise ValueError(kernel_type)

# KRLS

In [8]:
def calc_err(Ypred, Ytrue):
    return np.mean((Ypred != Ytrue))*100

In [9]:
def krls_train(x, y, reg_par, kernel_type, kernel_par):
    K = kernel_matrix(x, x, kernel_type, kernel_par)
    n = x.shape[0]
    cov = K + reg_par * n * np.eye(n)
    w = np.linalg.solve(cov, y)
    return w

def krls_predict(x_ts, x_tr, w, kernel_type, kernel_par):
    return np.sign(kernel_matrix(x_ts, x_tr, kernel_type, kernel_par) @ w)

In [10]:
def _make_kfold_indices(n_tot, num_folds, rng):
    perm = rng.choice(n_tot, size=n_tot, replace=False)
    return np.array_split(perm, num_folds)


def krls_kfold_valerr(x_tr, y_tr, split_idx, reg_par, kernel_type, kernel_par):
    """
    Compute train/val errors over *given* folds (split_idx is a list of index arrays).
    """
    n_tot = x_tr.shape[0]
    assert len(split_idx) > 1 and all(idx.dtype == int for idx in split_idx)
    assert sum(len(idx) for idx in split_idx) == n_tot

    tr_errs, val_errs = [], []
    for val_ids in split_idx:
        val_mask = np.zeros(n_tot, dtype=bool)
        val_mask[val_ids] = True

        kf_x_tr, kf_y_tr = x_tr[~val_mask], y_tr[~val_mask]
        kf_x_val, kf_y_val = x_tr[val_mask], y_tr[val_mask]

        w_krls   = krls_train(kf_x_tr, kf_y_tr, reg_par=reg_par, kernel_type=kernel_type, kernel_par=kernel_par)
        pred_tr  = krls_predict(kf_x_tr,  kf_x_tr, w_krls, kernel_type=kernel_type, kernel_par=kernel_par)
        pred_val = krls_predict(kf_x_val, kf_x_tr, w_krls, kernel_type=kernel_type, kernel_par=kernel_par)

        tr_errs.append(calc_err(pred_tr,  kf_y_tr))
        val_errs.append(calc_err(pred_val, kf_y_val))

    return np.asarray(tr_errs), np.asarray(val_errs)


def krls_kfoldcv(x_tr, y_tr, num_folds, reg_par_list, kernel_type, kernel_par_list, rng=None):
    """
    Choose the best parameters for both the regularizer and the kernel parameter according to K-Fold CV.
    """
    n_tot = x_tr.shape[0]
    assert num_folds > 1 and num_folds <= n_tot

    # Precompute shared folds once for fair comparison
    rng = _check_random_generator(rng)
    split_idx = _make_kfold_indices(n_tot, num_folds, rng)

    errors = np.zeros((len(reg_par_list), len(kernel_par_list)))
    for i, reg_par in enumerate(reg_par_list):
        for j, kernel_par in enumerate(kernel_par_list):
            _, val_error = krls_kfold_valerr(x_tr, y_tr, split_idx, reg_par, kernel_type, kernel_par)
            errors[i, j] = np.mean(val_error)

    best_i, best_j = np.unravel_index(np.argmin(errors), errors.shape)
    best_reg_par = reg_par_list[best_i]
    best_kernel_par = kernel_par_list[best_j]
    best_err = errors[best_i, best_j]

    print(f"The best error (MSE={best_err:.5f}) was obtained with "
          f"lambda={best_reg_par}, kernel parameter={best_kernel_par}")

    return best_reg_par, best_kernel_par, best_err